### Warning: This inference example contains content that can be offensive or upsetting.

In [1]:
import pandas as pd
from toxcl import ToXCL

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("t5-base")
tg_model = AutoModelForSeq2SeqLM.from_pretrained("saved/T5-TG")
decoder_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

# note that our choice: TG (t5-base) and decoder (flan-t5-base) models have the same tokenizer
# if one uses different backbone (e.g., bart for TG and t5 for decoder), he/she should load different tokenizers
toxcl_model = ToXCL(decoder_model, decoder_tokenizer=tokenizer, tg_model=tg_model, tg_tokenizer=tokenizer)
toxcl_model.load_checkpoint("saved/ToXCL")

/home/nhathm/miniconda3/envs/toxcl/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Successfully loaded checkpoint.


In [2]:
prompts = ["that is sick and racist", "you hate like a jew. you are fixatd on hating white christians like a jew", "she's another low iq hollywood liberal . islam is submission and btw if they ran america"]
cls_true = ["normal", "hate", "hate"]
expl_true = ["none", "Jewish people have hate toward other groups", "Librals are stupid"]

tg_generation_params = {
    "num_beams": 4,
    "top_p": 0.92,
    "max_new_tokens": 100,
    "do_sample": True
}
explanation_params = tg_generation_params.copy() # can be different if needed

outputs = toxcl_model.generate_e2e(
    prompts,
    apply_constraints=True,
    tg_generation_params=tg_generation_params,
    explanation_params=explanation_params
)

df = pd.DataFrame({
    "Prompts": prompts,
    "Ground-truth Class": cls_true,
    "Ground-truth Explanation": expl_true,
    "Target Groups": outputs["target_groups"],
    "Detection": outputs["detections"],
    "Explanations": outputs["explanations"]})
df

,Prompts,Ground-truth Class,Ground-truth Explanation,Target Groups,Detection,Explanations
0,that is sick and racist,normal,none,none,normal,none
1,you hate like a jew. you are fixatd on hating ...,hate,Jewish people have hate toward other groups,"Caucasian, Christian, Jewish",hate,Jews don't like while people
2,she's another low iq hollywood liberal . islam...,hate,Librals are stupid,"Islam, Women",hate,Liberals are low iq people
